**reshaping=转换表或者向量(DataFrame/Series)的结构，让其适合进行数据分析**

+ pivot
+ pivot-table
+ stack
+ un-stack

### pivot

pivot函数有三个参数 :

+ index
+ columns
+ values



In [19]:
from collections import OrderedDict
from pandas import DataFrame
import pandas as pd
import numpy as np

In [20]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item1', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1, 2, 3, 4])
))
df = DataFrame(table)
df

,Item,CType,USD,EU
0,Item0,Gold,1,1
1,Item0,Bronze,2,2
2,Item1,Gold,3,3
3,Item1,Silver,4,4


In [21]:
p=df.pivot(index='Item',columns='CType',values='USD')
p  #p里面没有EU信息，某种意义上pivot后是对原来信息的简化

CType,Bronze,Gold,Silver
Item,,,
Item0,2.0,1.0,NaN
Item1,NaN,3.0,4.0


In [19]:
# 原来的DataFrame: 获取Item0，金牌客户的的价值
print (df[(df.Item=='Item0') & (df.CType=='Gold')].USD.values)

[1]


In [20]:
# Pivoted DataFrame
print (p[p.index=='Item0'].Gold.values)

[ 1.]


### 多列pivot

In [22]:
p = df.pivot(index='Item', columns='CType')
p #muliindex ==hierarchical column

USD                 EU            
CType Bronze Gold Silver Bronze Gold Silver
Item                                       
Item0    2.0  1.0    NaN    2.0  1.0    NaN
Item1    NaN  3.0    4.0    NaN  3.0    4.0

In [23]:
print(df[(df.Item=='Item0') & (df.CType=='Gold')].USD.values)

[1]


In [24]:
print(p.USD[p.USD.index=='Item0'].Gold.values)

[ 1.]


### pivot常见错误-index/columns索引后对应后有多行相同值

In [40]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1, 2, 3, 4])
))
df = DataFrame(table)
print(df)
p = df.pivot(index='Item', columns='CType', values='USD')

    Item   CType  USD  EU
0  Item0    Gold    1   1
1  Item0  Bronze    2   2
2  Item0    Gold    3   3
3  Item1  Silver    4   4


ValueError: Index contains duplicate entries, cannot reshape

### pivot table

In [41]:
p=df.pivot_table(index='Item',columns='CType',values='USD')
p

CType,Bronze,Gold,Silver
Item,,,
Item0,2.0,2.0,NaN
Item1,NaN,NaN,4.0


In [42]:
df

,Item,CType,USD,EU
0,Item0,Gold,1,1
1,Item0,Bronze,2,2
2,Item0,Gold,3,3
3,Item1,Silver,4,4


In [46]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1.1, 2.2, 3, 4.4])
))
df = DataFrame(table)
p = df.pivot_table(index='Item', columns='CType', values='USD', aggfunc=np.min)  #aggfun指明aggretation函数
p

CType,Bronze,Gold,Silver
Item,,,
Item0,2.0,1.0,NaN
Item1,NaN,NaN,4.0


### stack/unstack

pivot实际是对DataFrame进行stack的一种特例。stack意味把最内层的列索引变成最内层的行索引，而unstack就是把最内层的行索引变列索引

![](./img/stack_unstack.png)

In [47]:
# Row Multi-Index
row_idx_arr = list(zip(['r0', 'r0'], ['r00', 'r01']))
row_idx = pd.MultiIndex.from_tuples(row_idx_arr)
print(row_idx)
# Column Multi-Index
col_idx_arr = list(zip(['c0', 'c0', 'c1'], ['c00', 'c01', 'c10']))
col_idx = pd.MultiIndex.from_tuples(col_idx_arr)
print(col_idx)
# Create the DataFrame
df = DataFrame(np.arange(6).reshape(2,3), index=row_idx, columns=col_idx)
df = df.applymap(lambda x: (x // 3, x % 3))
df

MultiIndex(levels=[['r0'], ['r00', 'r01']],
           labels=[[0, 0], [0, 1]])
MultiIndex(levels=[['c0', 'c1'], ['c00', 'c01', 'c10']],
           labels=[[0, 0, 1], [0, 1, 2]])


c0              c1
           c00     c01     c10
r0 r00  (0, 0)  (0, 1)  (0, 2)
   r01  (1, 0)  (1, 1)  (1, 2)

In [48]:
p=df.stack()
p

c0      c1
r0 r00 c00  (0, 0)     NaN
       c01  (0, 1)     NaN
       c10     NaN  (0, 2)
   r01 c00  (1, 0)     NaN
       c01  (1, 1)     NaN
       c10     NaN  (1, 2)

In [50]:
p.unstack()

c0                c1             
           c00     c01  c10  c00  c01     c10
r0 r00  (0, 0)  (0, 1)  NaN  NaN  NaN  (0, 2)
   r01  (1, 0)  (1, 1)  NaN  NaN  NaN  (1, 2)

### pivot案例

In [51]:
df = pd.read_excel("../data/sales-funnel.xlsx")
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [52]:
df.dtypes

Account      int64
Name        object
Rep         object
Manager     object
Product     object
Quantity     int64
Price        int64
Status      object
dtype: object

In [53]:
df["Status"] = df["Status"].astype("category")
df["Status"].cat.set_categories(["won","pending","presented","declined"],inplace=True)

In [54]:
df.dtypes

Account        int64
Name          object
Rep           object
Manager       object
Product       object
Quantity       int64
Price          int64
Status      category
dtype: object

In [6]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"])

Price
Manager       Rep                 
Debra Henley  Craig Booker   20000
              Daniel Hilton  38333
              John Smith     20000
Fred Anderson Cedric Moss    27500
              Wendy Yule     44250

In [7]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=np.sum)

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

In [8]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=[np.mean,len])

mean   len
                             Price Price
Manager       Rep                       
Debra Henley  Craig Booker   20000     4
              Daniel Hilton  38333     3
              John Smith     20000     2
Fred Anderson Cedric Moss    27500     4
              Wendy Yule     44250     4

In [9]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],
               columns=["Product"],aggfunc=[np.sum])

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

In [10]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],
               columns=["Product"],aggfunc=[np.sum],fill_value=0)

sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

In [11]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price","Quantity"],
               columns=["Product"],aggfunc=[np.sum],fill_value=0)

sum                                        \
                              Price                              Quantity   
Product                         CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                           
Debra Henley  Craig Booker    65000        5000       0    10000        2   
              Daniel Hilton  105000           0       0    10000        4   
              John Smith      35000        5000       0        0        1   
Fred Anderson Cedric Moss     95000        5000       0    10000        3   
              Wendy Yule     165000        7000    5000        0        7   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            2       0        1  
              Daniel Hilton           0       0        1  
              John Smith              2       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              3       2        0

In [12]:
pd.pivot_table(df,index=["Manager","Rep","Product"],
               values=["Price","Quantity"],aggfunc=[np.sum],fill_value=0)

sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2

In [13]:
pd.pivot_table(df,index=["Manager","Rep","Product"],
               values=["Price","Quantity"],
               aggfunc=[np.sum,np.mean],fill_value=0,margins=True)

sum                   mean  \
                                            Price Quantity         Price   
Manager       Rep           Product                                        
Debra Henley  Craig Booker  CPU           65000.0      2.0  32500.000000   
                            Maintenance    5000.0      2.0   5000.000000   
                            Software      10000.0      1.0  10000.000000   
              Daniel Hilton CPU          105000.0      4.0  52500.000000   
                            Software      10000.0      1.0  10000.000000   
              John Smith    CPU           35000.0      1.0  35000.000000   
                            Maintenance    5000.0      2.0   5000.000000   
Fred Anderson Cedric Moss   CPU           95000.0      3.0  47500.000000   
                            Maintenance    5000.0      1.0   5000.000000   
                            Software      10000.0      1.0  10000.000000   
              Wendy Yule    CPU          165000.0      7.0  82500.000000   
                            Maintenance    7000.0      3.0   7000.000000   
                            Monitor        5000.0      2.0   5000.000000   
All                                      522000.0     30.0  30705.882353   

                                                   
                                         Quantity  
Manager       Rep           Product                
Debra Henley  Craig Booker  CPU          1.000000  
                            Maintenance  2.000000  
                            Software     1.000000  
              Daniel Hilton CPU          2.000000  
                            Software     1.000000  
              John Smith    CPU          1.000000  
                            Maintenance  2.000000  
Fred Anderson Cedric Moss   CPU          1.500000  
                            Maintenance  1.000000  
                            Software     1.000000  
              Wendy Yule    CPU          3.500000  
                            Maintenance  3.000000  
                            Monitor      2.000000  
All                                      1.764706

In [14]:
pd.pivot_table(df,index=["Manager","Status"],values=["Price"],
               aggfunc=[np.sum],fill_value=0,margins=True)

sum
                            Price
Manager       Status             
Debra Henley  won         65000.0
              pending     50000.0
              presented   50000.0
              declined    70000.0
Fred Anderson won        172000.0
              pending      5000.0
              presented   45000.0
              declined    65000.0
All                      522000.0

In [15]:
pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc={"Quantity":len,"Price":np.sum},fill_value=0)

Quantity                                Price  \
Product                      CPU Maintenance Monitor Software     CPU   
Manager       Status                                                    
Debra Henley  won              1           0       0        0   65000   
              pending          1           2       0        0   40000   
              presented        1           0       0        2   30000   
              declined         2           0       0        0   70000   
Fred Anderson won              2           1       0        0  165000   
              pending          0           1       0        0       0   
              presented        1           0       1        1   30000   
              declined         1           0       0        0   65000   

                                                      
Product                 Maintenance Monitor Software  
Manager       Status                                  
Debra Henley  won                 0       0        0  
              pending         10000       0        0  
              presented           0       0    20000  
              declined            0       0        0  
Fred Anderson won              7000       0        0  
              pending          5000       0        0  
              presented           0    5000    10000  
              declined            0       0        0

In [55]:
table = pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc={"Quantity":len,"Price":[np.sum,np.mean]},fill_value=0)
table

Quantity                               Price  \
                             len                                mean   
Product                      CPU Maintenance Monitor Software    CPU   
Manager       Status                                                   
Debra Henley  won              1           0       0        0  65000   
              pending          1           2       0        0  40000   
              presented        1           0       0        2  30000   
              declined         2           0       0        0  35000   
Fred Anderson won              2           1       0        0  82500   
              pending          0           1       0        0      0   
              presented        1           0       1        1  30000   
              declined         1           0       0        0  65000   

                                                                          \
                                                         sum               
Product                 Maintenance Monitor Software     CPU Maintenance   
Manager       Status                                                       
Debra Henley  won                 0       0        0   65000           0   
              pending          5000       0        0   40000       10000   
              presented           0       0    10000   30000           0   
              declined            0       0        0   70000           0   
Fred Anderson won              7000       0        0  165000        7000   
              pending          5000       0        0       0        5000   
              presented           0    5000    10000   30000           0   
              declined            0       0        0   65000           0   

                                          
                                          
Product                 Monitor Software  
Manager       Status                      
Debra Henley  won             0        0  
              pending         0        0  
              presented       0    20000  
              declined        0        0  
Fred Anderson won             0        0  
              pending         0        0  
              presented    5000    10000  
              declined        0        0

In [17]:
table.query('Manager == ["Debra Henley"]')

Quantity                               Price  \
                            len                                mean   
Product                     CPU Maintenance Monitor Software    CPU   
Manager      Status                                                   
Debra Henley won              1           0       0        0  65000   
             pending          1           2       0        0  40000   
             presented        1           0       0        2  30000   
             declined         2           0       0        0  35000   

                                                                        \
                                                       sum               
Product                Maintenance Monitor Software    CPU Maintenance   
Manager      Status                                                      
Debra Henley won                 0       0        0  65000           0   
             pending          5000       0        0  40000       10000   
             presented           0       0    10000  30000           0   
             declined            0       0        0  70000           0   

                                         
                                         
Product                Monitor Software  
Manager      Status                      
Debra Henley won             0        0  
             pending         0        0  
             presented       0    20000  
             declined        0        0

In [18]:
table.query('Status == ["pending","won"]')

Quantity                               Price  \
                           len                                mean   
Product                    CPU Maintenance Monitor Software    CPU   
Manager       Status                                                 
Debra Henley  won            1           0       0        0  65000   
              pending        1           2       0        0  40000   
Fred Anderson won            2           1       0        0  82500   
              pending        0           1       0        0      0   

                                                                        \
                                                       sum               
Product               Maintenance Monitor Software     CPU Maintenance   
Manager       Status                                                     
Debra Henley  won               0       0        0   65000           0   
              pending        5000       0        0   40000       10000   
Fred Anderson won            7000       0        0  165000        7000   
              pending        5000       0        0       0        5000   

                                        
                                        
Product               Monitor Software  
Manager       Status                    
Debra Henley  won           0        0  
              pending       0        0  
Fred Anderson won           0        0  
              pending       0        0